Original https://ts.gluon.ai/stable/tutorials/forecasting/quick_start_tutorial.html

In [ ]:
# Установка библиотек
!pip install --upgrade mxnet
!pip install gluonts
# необходима для ускорения работы с json
!pip install orjson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 3.8 MB/s eta 0:00:00


In [ ]:
# Подключение библиотек
import gluonts
import orjson
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
import numpy as np
import pandas as pd
from gluonts.dataset.common import ListDataset

In [ ]:
# Формирование датасета с синтетическими данными
n_param = 10
t_x = 100
prediction_length = 24
freq = "1H"
custom_dataset = np.random.normal(size=(n_param, t_x))
custom_dataset[:, :10]

array([[ 1.68069246e+00,  5.74003070e-01,  1.52479149e+00,
         3.19004636e-02,  2.81632431e-01,  1.58652970e+00,
        -6.92973181e-01,  5.63699597e-01,  9.68303898e-01,
         1.73423124e+00],
       [ 6.57734094e-01,  7.19727562e-01,  4.25360129e-01,
        -2.36173562e-01, -1.65090383e+00, -7.85522163e-01,
        -7.21579340e-01, -3.49592079e-02, -1.88068711e+00,
         8.92755227e-01],
       [-9.51790918e-01, -4.58778136e-01, -3.22186314e-02,
         5.42216000e-01, -2.23946029e-01, -1.63267266e+00,
        -1.11794393e-01,  1.01837388e+00, -1.85264626e-01,
         1.00322485e-01],
       [-1.05951004e+00,  1.33038718e+00, -8.55010410e-01,
         8.87951606e-01, -2.55805410e-01,  1.94788541e+00,
        -5.88321087e-01, -5.99901016e-02, -6.66615464e-01,
        -4.11369191e-01],
       [-5.59564176e-01,  6.70617420e-01,  4.32866603e-01,
         4.99274524e-01,  3.90392817e-01,  1.21583843e+00,
        -2.45289715e+00, -8.25874245e-01,  1.63869638e+00,
        -9.

In [ ]:
start = pd.Period("01-01-2019", freq=freq)
# Обучающая выборка
ls_train = [{"target": x, "start": start} for x in custom_dataset[:,
                                                          :-prediction_length]]
train_ds = ListDataset(ls_train, freq=freq)
len(train_ds[4].get('target')), train_ds[4].get('start')

(76, Period('2019-01-01 00:00', 'H'))

In [ ]:
# test dataset: use the whole dataset, add "target" and "start" fields
test_ds = ListDataset(
    [{"target": x, "start": start} for x in custom_dataset], freq=freq
)
trainer = Trainer(epochs=20, learning_rate=1e-3)
# batch_size=512, patience=5
estimator = DeepAREstimator(freq=freq, context_length=24 * 5, ## context length is number of time steps will look back here it is 5 days
                            prediction_length=prediction_length, cardinality=[1],
                            num_layers=2, dropout_rate = 0.25)
#use_feat_static_cat=True, num_cells=32, cell_type='lstm', trainer=trainer
predictor = estimator.train(train_ds)